In [20]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
import os
import matplotlib.pyplot as plt
from transformers import AlbertTokenizerFast, AlbertModel
# from gensim.models import Word2Vec
# from nets import *

In [21]:
'''CONSTANTS'''
batch_size = 32
max_epoch = 40
lr = 0.001

LD = 30
LW = 20

device = torch.device(
    'cuda') if torch.cuda.is_available() else torch.device('cpu')
train_data_path = '../../../Datasets/Model/train'
test_data_path = '../../../Datasets/Model/test'
image_path = '../../../Datasets/Images'
data_save_path = '../../../Save/Experiment5'
analytics_path = '../../../Analysis/Experiment5'
nlp_path = '../../../Datasets/Model/nlp'

'''NLP'''
# load word2vec models from nlp_path
# model_review = Word2Vec.load(f'{nlp_path}/model_review.model')
# model_title = Word2Vec.load(f'{nlp_path}/model_title.model')

# get the vocabulary of reviews titles
# vocab_review = model_review.wv.index_to_key
# vocab_title = model_title.wv.index_to_key

# # get the vectors of reviews and titles
# vecs_review = torch.tensor(model_review.wv[vocab_review])
# vecs_title = torch.tensor(model_title.wv[vocab_title])

# get the glove embeddings
# print("Loading glove embeddings...")

'''ANALYTICS'''

rmse_arr = []


In [10]:
train_users = np.load(f'{train_data_path}/train_users.npy')
train_items = np.load(f'{train_data_path}/train_items.npy')
train_ratings = np.load(f'{train_data_path}/train_ratings.npy')
train_reviews = np.load(
    f'{train_data_path}/train_reviews.npy', allow_pickle=True)
train_descriptions = np.load(
    f'{train_data_path}/train_descriptions.npy', allow_pickle=True)
train_titles = np.load(
    f'{train_data_path}/train_titles.npy', allow_pickle=True)
train_prices = np.load(f'{train_data_path}/train_prices.npy')
train_categories = np.load(f'{train_data_path}/train_categories.npy')

test_users = np.load(f'{test_data_path}/test_users.npy')
test_items = np.load(f'{test_data_path}/test_items.npy')
test_ratings = np.load(f'{test_data_path}/test_ratings.npy')
test_reviews = np.load(f'{test_data_path}/test_reviews.npy', allow_pickle=True)
test_descriptions = np.load(
    f'{test_data_path}/test_descriptions.npy', allow_pickle=True)
test_titles = np.load(
    f'{test_data_path}/test_titles.npy', allow_pickle=True)
test_prices = np.load(f'{test_data_path}/test_prices.npy')
test_categories = np.load(f'{test_data_path}/test_categories.npy')


In [11]:
# this function accepts a sequence, and pads the setences in the sequence to LW length, then returns the padded sequence
def pad_sequence(sequence, lw = LW, ld = LD, placeholder=0):
    arr = []
    for sentence in sequence:
      if len(sentence) < lw:
        arr.append(sentence + [placeholder] * (lw - len(sentence)))
      else:
        arr.append(sentence[:lw])

    if len(arr) > ld:
      arr = arr[:ld]
    else:
      for i in range (ld-len(arr)):
        arr.append([placeholder] * lw)
        
    return arr

'''sample user'''
'''this function takes a user, who is from a set of users, a set of items, a set of reviews, a set of titles, 
a set of categories, a set of prices, and a set of ratings.'''
def sample_user(user, users, items, reviews, ratings):
  # find out all the indicies of the user in users
  user_indices = np.where(users == user)[0]
  # randomly select an index from these indicies.
  user_index = np.random.choice(user_indices)

  # get the item that the user has purchased.
  item = items[user_index]

  # get the rating that the user has given to that item.
  rating = ratings[user_index]

  # find out the indices of the item in items
  item_indices = np.where(items == item)[0]

  # get the reviews of that item
  item_reviews = [reviews[i] for i in item_indices if i!= user_index]
  user_reviews = [reviews[i] for i in user_indices if i!= user_index]

  user_reviews = pad_sequence(user_reviews)
  item_reviews = pad_sequence(item_reviews)

  return user, item, rating, item_reviews, user_reviews


'''function to get a batch of training samples.
this function selects a random user from a set of users
and gets his information by function sample_user, repeat it for batch_size times and 
gets the batch of samples. Each element in the batch tuple is then transormed to a pytorch tensor and returned.'''
def get_batch(users, items, reviews, ratings, batch_size=32, fixed_users_set = None):
  # get a batch of users
  if fixed_users_set is None:
    batch_users = np.random.choice(users, size=batch_size)
  else:
    batch_users = fixed_users_set
  # get the batch of samples
  batch = [list(sample_user(user, users, items, reviews, ratings)) for user in batch_users]
  # transform each column of the batch to a pytorch tensor
  batch = [torch.tensor(sample) for sample in zip(*batch)]
  return batch



In [7]:
# get a batch of training samples
batch = get_batch(train_users, train_items, train_reviews, train_ratings)
user, item, rating, item_reviews, user_reviews = batch

In [22]:
def weights_init(m):
  if isinstance(m, nn.Linear):
    nn.init.normal_(m.weight.data)
    nn.init.constant_(m.bias.data, 0.0)


class GMF(nn.Module):
    def __init__(self, inp_range, latent_dim=20, dropout=True):
        super().__init__()
        self.emb = nn.Embedding(
            num_embeddings=inp_range, embedding_dim=latent_dim)
        self.dropout = nn.Dropout(0.5)
        self.use_dropout = dropout

    def forward(self, inputs):
        embedding = self.emb(inputs)
        if self.use_dropout:
            embedding = self.dropout(embedding)

        return embedding

class FM(nn.Module):
    def __init__(self, latent_dim, fea_num):
        super().__init__()

        self.latent_dim = latent_dim
        self.w0 = nn.Parameter(torch.zeros([1, ], requires_grad=True))
        self.w1 = nn.Parameter(torch.rand([fea_num, 1], requires_grad=True))
        self.w2 = nn.Parameter(torch.rand([fea_num, latent_dim], requires_grad=True))

    def forward(self, inputs):
        # inputs = inputs.long()
        first_order = self.w0 + torch.mm(inputs, self.w1)
        second_order = 1/2 * torch.sum(
            torch.pow(torch.mm(inputs, self.w2), 2) -
            torch.mm(torch.pow(inputs, 2), torch.pow(self.w2, 2)),

            dim=1,
            keepdim=True
        )

        return first_order + second_order


class MPCNExt(nn.Module):
  def __init__(self, embed_size = 50, lw = LW, fm_dim = 8, np = 1):
      super().__init__()
      self.np = np
      self.embedding = nn.Embedding(100000, embed_size)
      # initialise a weighting matrix of size (50 x 50)
      self.W_g = nn.Parameter(torch.rand(embed_size, embed_size, requires_grad=True) * 0.001)
      # initialise a bias vector of size (50)
      self.b_g = nn.Parameter(torch.zeros(embed_size, requires_grad=True))
      # initialise a weighting matrix of size (50 x 50)
      self.W_u = nn.Parameter(torch.rand(embed_size, embed_size, requires_grad=True) * 0.001)
      # initialise a bias vector of size (50)
      self.b_u = nn.Parameter(torch.zeros(embed_size, requires_grad=True))
      self.sigmoid = nn.Sigmoid()
      self.tanh = nn.Tanh()

      self.F_review = nn.Sequential(
        nn.Linear(embed_size, 2*embed_size, bias=True),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(2*embed_size, embed_size, bias=True),
        nn.ReLU(),
        nn.Dropout(0.2),
      )

      self.F_word = nn.Sequential(
        nn.Linear(embed_size, 2*embed_size, bias=True),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(2*embed_size, embed_size, bias=True),
        nn.ReLU(),
        nn.Dropout(0.2),
      )

      self.pointer_nn = nn.Linear(np * embed_size, embed_size,bias=True)

      # initialise a weighting matrix of size (50 x 50)
      self.M = nn.Parameter(torch.rand(embed_size, embed_size, requires_grad=True) * 0.001)
      self.M_w = nn.Parameter(torch.rand(embed_size, embed_size, requires_grad=True) * 0.001)

      self.fm = FM(fm_dim, 2 * embed_size + 50)

  def forward(self, a, b, meta=None):
    a = self.embedding(a.long())
    b = self.embedding(b.long())
    # size of a: (32, 30, 20, 50)
    # size of b: (32, 30, 20, 50)

    a_sumed = torch.sum(a, dim=2)
    b_sumed = torch.sum(b, dim=2)
    # size of a_sumed: (32, 30, 50)
    # size of b_sumed: (32, 30, 50)

    a_bar = self.sigmoid(a_sumed @ self.W_u) + self.b_g * self.tanh(a_sumed @ self.W_g + self.b_u)
    b_bar = self.sigmoid(b_sumed @ self.W_u) + self.b_g * self.tanh(b_sumed @ self.W_g + self.b_u)
    # size of a_bar: (32, 30, 50)
    # size of b_bar: (32, 30, 50)

    a_prime_arr = []
    b_prime_arr = []

    for i in range (self.np):
      S = self.F_review(a_bar) @ self.M @ self.F_review(b_bar).permute(0,2,1)
      # get the maximum of S columnwise
      max_col_S = torch.max(S, dim=1)[0]
      # get the maximum of S rowwise
      max_row_S = torch.max(S, dim=2)[0]

      # size of S: (32, 30, 30)
      # size of max_col_S: (32, 30)
      # size of max_row_S: (32, 30)

      p_a = nn.functional.gumbel_softmax(logits=max_col_S, hard=True, dim=-1, tau=0.1).unsqueeze(1).unsqueeze(2)
      p_b = nn.functional.gumbel_softmax(logits=max_row_S, hard=True, dim=-1, tau=0.1).unsqueeze(1).unsqueeze(2)
      # size of p_a: (32, 1, 1, 30)
      # size of p_b: (32, 1, 1, 30)

      ''' This step implements the pointer selection mechanism in the paper 
      
      p_a is originally a two-dimensional tensor of size (32 x 30(LW)), while a, b \
      are originally a four-dimensional tensor of size (32 x 30(LW) x 20 x 50) \
      we need to reshape p_a and p_b to a four-dimensional tensor of size (32, 1, 1, 30) \
      and multiply with reshaped a and b to (32, 20, 30, 50) (a.permute(0,2,1,3)) \
      then we get the correct but not ideally-shaped selected review of size a' = (32, 1, 20, 50) \
      then we squeeze the second dimension (a'.squeeze(1)) of a' to get (32, 20, 50)
      
      ''' 

      a_prime = (p_a @ a.permute(0,2,1,3)).permute(0,2,1,3).squeeze(1)
      b_prime = (p_b @ b.permute(0,2,1,3)).permute(0,2,1,3).squeeze(1)
      # size of a_prime: (32, lw, 50)
      # size of b_prime: (32, lw, 50)

      a_prime_arr.append(a_prime)
      b_prime_arr.append(b_prime)

    # concatenate the elements of a_prime_arr and b_prime_arr
    a_prime = torch.cat(a_prime_arr, dim=1)
    b_prime = torch.cat(b_prime_arr, dim=1)

    
    W = self.F_word(a_prime) @ self.M_w @ self.F_word(b_prime).permute(0,2,1)
    # size of W: (batch_size, lw * np, lw * np)

    # get the column average of W
    avg_col_W = torch.mean(W, dim=1).unsqueeze(2)
    # get the row average of W
    avg_row_W = torch.mean(W, dim=2).unsqueeze(2)
    # size of avg_col_W: (batch_size, lw * np, 1)
    # size of avg_row_W: (batch_size, lw * np, 1)
    
    a_bar_prime = self.sigmoid(avg_col_W).permute(0,2,1) @ a_prime
    b_bar_prime = self.sigmoid(avg_row_W).permute(0,2,1) @ a_prime
    # size of a_bar_prime: (batch_size, 1, emb_size)
    # size of b_bar_prime: (batch_size, 1, emb_size)

    a_bar_prime = a_bar_prime.squeeze(1)
    b_bar_prime = b_bar_prime.squeeze(1)

    # concatenate a_bar_prime and b_bar_prime
    a_bar_prime_bar_prime = torch.cat((a_bar_prime, b_bar_prime), dim=1)

    # concat meta and a_bar_prime_bar_prime
    if meta is not None:
      z = torch.cat((a_bar_prime_bar_prime, meta), dim=1)
      pred = self.fm(z)
    else:
      pred = self.fm(a_bar_prime_bar_prime)

    # flatten the predictions
    pred = pred.squeeze(1)

    return pred

In [23]:
mpcn = MPCNExt().apply(weights_init).to(device)
optimiser_mpcn = optim.Adam(mpcn.parameters(), lr=lr, weight_decay=1e-6)

mf_u = GMF(max(train_users)+1, 25, True).apply(weights_init).to(device)
# item latent vectors -> this embedding should be updated
mf_i = GMF(max(max(train_items), max(test_items))+1,
           25, True).apply(weights_init).to(device)

optimiser_mf_u = optim.Adam(mf_u.parameters(), lr=lr, weight_decay=1e-6)
optimiser_mf_i = optim.Adam(mf_i.parameters(), lr=lr, weight_decay=1e-6)

def save_training(path = f'{data_save_path}/'):
  torch.save(mpcn.state_dict(), path + 'mpcn_p1_ext.pt')
  torch.save(mf_u.state_dict(), path + 'mf_u_p1_ext.pt')
  torch.save(mf_i.state_dict(), path + 'mf_i_p1_ext.pt')
  torch.save(optimiser_mpcn.state_dict(), path + 'optimiser_mpcn_p1_ext.pt')
  torch.save(optimiser_mf_u.state_dict(), path + 'optimiser_mf_u_p1_ext.pt')
  torch.save(optimiser_mf_i.state_dict(), path + 'optimiser_mf_i_p1_ext.pt')
  print('Saved model and optimiser')

def load_training(path = f'{data_save_path}/'):
  mpcn.load_state_dict(torch.load(path + 'mpcn_p1_ext.pt'))
  mf_u.load_state_dict(torch.load(path + 'mf_u_p1_ext.pt'))
  mf_i.load_state_dict(torch.load(path + 'mf_i_p1_ext.pt'))
  optimiser_mpcn.load_state_dict(torch.load(path + 'optimiser_mpcn_p1_ext.pt'))
  optimiser_mf_u.load_state_dict(torch.load(path + 'optimiser_mf_u_p1_ext.pt'))
  optimiser_mf_i.load_state_dict(torch.load(path + 'optimiser_mf_i_p1_ext.pt'))
  print('Loaded model and optimiser')
  

In [24]:
'''Training'''
RMSE = []
best_rmse = 1e6

def evaluation(test_batch_size = 32):
  test_users_unique = list(set(test_users))
  rmse_arr = []
  for i in tqdm(range(0, len(test_users_unique), test_batch_size)):
    test_users_batch = test_users_unique[i:i+test_batch_size]
    batch_test = get_batch(test_users, test_items, test_reviews, test_ratings, fixed_users_set=test_users_batch)
    user, item, rating, item_reviews, user_reviews = batch_test
    user, item, rating, item_reviews, user_reviews = user.to(device), item.to(device), rating.to(device), item_reviews.to(device), user_reviews.to(device)

    latent_user = mf_u(user)
    latent_item = mf_i(item)

    # concatenate latent_user and latent_item
    meta = torch.cat((latent_user, latent_item), dim=1)

    pred = mpcn(item_reviews, user_reviews, meta)

    # calculate the loss
    loss = torch.mean((pred - rating)**2)

    # calculate rooted mean square error
    rmse_arr.append(torch.sqrt(torch.mean((pred - rating)**2)).item())
  
  # return the mean of the rmse_arr
  return np.mean(rmse_arr)

print("Program: Evaluating the baseline RMSE of the model on the test set")

# set best_rmse to be the largest possible value
best_rmse = evaluation()
RMSE.append(best_rmse)

# print baseline best_rmse
print('Baseline RMSE:', best_rmse)

for epoch in range (max_epoch):
  for i in tqdm(range(1000)):
    batch =  get_batch(train_users, train_items, train_reviews, train_ratings)
    user, item, rating, item_reviews, user_reviews = batch
    user, item, rating, item_reviews, user_reviews = user.to(device), item.to(device), rating.to(device), item_reviews.to(device), user_reviews.to(device)

    latent_user = mf_u(user)
    latent_item = mf_i(item)

    # concatenate latent_user and latent_item
    meta = torch.cat((latent_user, latent_item), dim=1)

    pred = mpcn(item_reviews, user_reviews, meta)

    # calculate the loss
    loss = torch.mean((pred - rating)**2)

    # backpropagation
    optimiser_mpcn.zero_grad()
    optimiser_mf_u.zero_grad()
    optimiser_mf_i.zero_grad()
    loss.backward()
    optimiser_mpcn.step()
    optimiser_mf_u.step()
    optimiser_mf_i.step()

  '''evaluate the model on the test set'''
  rmse_test = evaluation()
  RMSE.append(rmse_test)
  # print out the loss of the models and the test rmse
  print('Epoch:', epoch, 'Loss:', loss.item(), 'Test RMSE:', rmse_test)
  if rmse_test < best_rmse:
    best_rmse = rmse_test
    save_training()


Program: Evaluating the baseline RMSE of the model on the test set


100%|██████████| 3609/3609 [01:21<00:00, 44.21it/s]


Baseline RMSE: 29818.224418337835


100%|██████████| 3609/3609 [01:21<00:00, 44.16it/s]


Epoch: 0 Loss: 7762.63037109375 Test RMSE: 45.84044692984708
Saved model and optimiser


...

Epoch: 37 Loss: 1.8532681465148926 Test RMSE: 1.15645134297062


100%|██████████| 3609/3609 [01:25<00:00, 42.15it/s]


Epoch: 38 Loss: 1.7138607501983643 Test RMSE: 1.1541377669161592


100%|██████████| 3609/3609 [01:23<00:00, 43.40it/s]


Epoch: 39 Loss: 0.9306067228317261 Test RMSE: 1.1520691911544652
Saved model and optimiser


In [25]:
with open (f'{analytics_path}/rmse_p1_ext.txt', 'w') as f:
  for r in RMSE:
    f.write(str(np.round(r, 4)) + '\n')